In [ ]:
import requests
import json
import sys
import re
import pandas as pd
import time

# Functions to Use

In [ ]:
def request_pockets(pdb_id, chain_id):
    # Data to be sent in the POST request
    data = {
        "dogsite": {
            "pdbCode": pdb_id,
            "analysisDetail": "1",
            "bindingSitePredictionGranularity": "1",
            "ligand": "",
            "chain": chain_id
        }
    }

    # Headers for the request
    headers = {"Accept": "application/json",
               "Content-Type": "application/json"}

    # URL to send the request
    url = "https://proteins.plus/api/dogsite_rest"
    
    # Sending POST request
    response = requests.post(url, data=json.dumps(data), headers=headers)

    # Checking the response
    counter = 1
    while response.status_code != 200:
        if response.status_code == 202:
            # Jobs run for a while. Adjust the time interval as needed
            wait_time = 120
            print(f'Job is running. Gonna wait! Waiting for: {counter*wait_time} seconds')
            time.sleep(wait_time)
            response = requests.post(url, data=json.dumps(data), headers=headers)
            counter += 1
            print("Response:", response.json())
        
        elif response.status_code == 400:
            print("Request failed with status code:", response.status_code)
            print("Request failed with error:", response.error) 
            print("Request failed with message:", response.message) 
            break
    
        elif response.status_code == 429:
            print(f"Too many requests, run this pdb again: {pdb_id}")
            print("Request failed with status code:", response.status_code)
            print("Request failed with error:", response.error) 
            print("Request failed with message:", response.message)
            break
            
        else:
            print(f"Unknown error for: {pdb_id}")
            print("Request failed with status code:", response.status_code)
            print("Request failed with error:", response.error) 
            print("Request failed with message:", response.message)
            break
            
    if response.status_code == 200:
        print("Request successful!")
        print("Response:", response.json())
    
    return response

In [ ]:
def extract_pocket(pdb):
    # Pattern to extract floats after 'Geometric pocket center at' and 'with max radius'
    pattern = r"Geometric pocket center at\s+([-+]?\d*\.\d+\s+[-+]?\d*\.\d+\s+[-+]?\d*\.\d+)\swith max radius\s+([-+]?\d*\.\d+)"

    # Finding matches in the text
    matches = re.search(pattern, pdb)

    if matches:
        # Extracting floats and returning as a list
        x_coor = matches.group(1).split()[0]
        y_coor = matches.group(1).split()[1]
        z_coor = matches.group(1).split()[2]
        max_radius = matches.group(2)
        return x_coor, y_coor, z_coor, max_radius
    else:
        x_coor, y_coor, z_coor, max_radius = None, None, None, None
        return x_coor, y_coor, z_coor, max_radius

# Example Use for One Pdb/Chain

In [ ]:
# Pdb id to look for the pockets
pdb_id, chain_id = "1kzk", "A"

In [ ]:
response = request_pockets(pdb_id, chain_id)
job_id = response.json()["location"]
# Get the residues
pdb_list = requests.get(job_id).json()["residues"]
# Get results table, all pockets are here
result_table = requests.get(job_id).json()["result_table"]
results_table_text = requests.get(result_table).text.split("\n")

df_pocket = pd.DataFrame(results_table_text[1:-1])[0].str.split("\t",expand=True)
header = results_table_text[0].split("\t")
df_pocket.columns = header
df_pocket

In [ ]:
# Add x, y, and z coordinates, and radius of the pocket for all pockets
for pdb in pdb_list:
    start_index = pdb.find(pdb_id + "_") + len(pdb_id + "_")
    end_index = pdb.find("_res.pdb")
    pocket_name = pdb[start_index:end_index]
    
    pdb_text = requests.get(pdb).text
    x_coor, y_coor, z_coor, max_radius = extract_pocket(pdb_text)

    df_pocket.loc[df_pocket["name"]==pocket_name, "x_coor"] = x_coor
    df_pocket.loc[df_pocket["name"]==pocket_name, "y_coor"] = y_coor
    df_pocket.loc[df_pocket["name"]==pocket_name, "z_coor"] = z_coor

    df_pocket.loc[df_pocket["name"]==pocket_name, "max_radius"] = max_radius

In [ ]:
df_pocket